<a href="https://colab.research.google.com/github/Polimolina/METEO/blob/main/Emergencias_99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install streamlit
!pip install streamlit_folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 6.4 MB/s eta 0:00:00


In [8]:
import streamlit as st
import pandas as pd
import requests
import folium
from streamlit_folium import folium_static

# Funciones de obtención de datos reales

@st.cache_data(ttl=600)
def get_earthquake_data():
    url = "https://www.ign.es/web/resources/sismologia/sismos/provincia.json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        earthquakes = []
        for item in data:
            earthquakes.append({
                "fecha": item["fecha"],
                "hora": item["hora"],
                "magnitud": item["magnitud"],
                "profundidad": item["profundidad"],
                "latitud": item["latitud"],
                "longitud": item["longitud"],
                "localizacion": item["provincia"]
            })
        return pd.DataFrame(earthquakes)
    except Exception as e:
        st.error(f"Error al obtener datos de terremotos: {e}")
        return pd.DataFrame()

@st.cache_data(ttl=600)
def get_weather_data(aemet_api_key):
    base_url = "https://opendata.aemet.es/opendata/api/observacion/convencional/todas/"
    headers = {"accept": "application/json"}

    try:
        response = requests.get(base_url, headers=headers, params={"api_key": aemet_api_key})
        response.raise_for_status()
        json_url = response.json()["datos"]

        data_response = requests.get(json_url)
        data_response.raise_for_status()
        weather_data = data_response.json()

        weather = []
        for obs in weather_data:
            weather.append({
                "fecha": obs["fint"],
                "latitud": obs["lat"],
                "longitud": obs["lon"],
                "temperatura": obs.get("ta", "N/A"),
                "humedad": obs.get("hr", "N/A"),
                "presion": obs.get("pres", "N/A"),
                "provincia": obs.get("ubi", "Desconocida")
            })

        return pd.DataFrame(weather)

    except Exception as e:
        st.error(f"Error al obtener datos meteorológicos de AEMET: {e}")
        return pd.DataFrame()

# Función de visualización (ejemplo)
def plot_emergency_map():
    mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=6)

    terremotos = get_earthquake_data()
    for _, row in terremotos.iterrows():
        folium.CircleMarker(
            [row["latitud"], row["longitud"]],
            radius=5,
            color='red',
            fill=True,
            fill_opacity=0.7,
            popup=f"Terremoto {row['magnitud']}M en {row['localizacion']}"
        ).add_to(mapa)

    folium_static(mapa)


# Interfaz Streamlit
st.title("Monitor de Emergencias en España")

# Input para la API Key (esto es lo que hace segura la compartición de código)
aemet_api_key = st.text_input("Introduce tu clave de API de AEMET", type="password")

if aemet_api_key:
    weather_data = get_weather_data(aemet_api_key)

    # Aquí puedes mostrar datos meteorológicos si quieres
    if not weather_data.empty:
        st.write("Datos meteorológicos recientes:")
        st.dataframe(weather_data)

    plot_emergency_map()
else:
    st.warning("Por favor, introduce la clave de API para obtener los datos meteorológicos.")



2025-03-04 18:33:35.703 No runtime found, using MemoryCacheStorageManager
2025-03-04 18:33:35.706 No runtime found, using MemoryCacheStorageManager
2025-03-04 18:33:35.709 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:33:35.710 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:33:35.711 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:33:35.713 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:33:35.714 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:33:35.715 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:33:35.715 Session state does not function when running a script without `streamlit

In [11]:
!pip install meteostat
import streamlit as st
import pandas as pd
from meteostat import Point, Daily
from datetime import datetime
import folium
from streamlit_folium import folium_static

# ============================
# 3. CONFIGURAR PERÍODO Y CIUDADES
# ============================
start = datetime(2018, 1, 1)
end = datetime.today()

# Lista de ciudades con coordenadas
ciudades = {
    "Madrid": (40.4168, -3.7038),
    "Valencia": (39.4699, -0.3763),
    "Barcelona": (41.3784, 2.1926),
    "Sevilla": (37.3891, -5.9845)
}

# Función para obtener las lluvias extremas y mostrarlas en el mapa
def get_extreme_rain_data():
    # DataFrame vacío para guardar resultados
    df_lluvias_extremas = pd.DataFrame()

    for ciudad, (lat, lon) in ciudades.items():
        st.write(f"📡 Descargando datos de {ciudad}...")

        # Definir punto y descargar datos
        punto = Point(lat, lon)
        df = Daily(punto, start, end).fetch()

        # Añadir metadatos
        df['ciudad'] = ciudad
        df['lat'] = lat
        df['lon'] = lon

        # Filtrar solo días con más de 100 mm de precipitación
        df_extrema = df[df['prcp'] > 100]

        # Añadir al DataFrame final
        df_lluvias_extremas = pd.concat([df_lluvias_extremas, df_extrema])

    return df_lluvias_extremas

# Función para mostrar los datos en un mapa
def plot_extreme_rain_map(df_lluvias_extremas):
    mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=5)

    for _, row in df_lluvias_extremas.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=10,
            color='blue',
            fill=True,
            fill_opacity=0.7,
            popup=(
                f"Ciudad: {row['ciudad']}<br>"
                f"Fecha: {row['time'].date()}<br>"
                f"Precipitación: {row['prcp']} mm"
            )
        ).add_to(mapa)

    # Mostrar mapa en Streamlit
    folium_static(mapa)

# ===================================
# INTERFAZ PRINCIPAL DE STREAMLIT
# ===================================
st.title("Monitor de Lluvias Extremas en España")

# Preguntar si se quiere cargar los datos de lluvias extremas
if st.button("Cargar datos de lluvias extremas en España"):
    # Obtener los datos de lluvias extremas
    df_lluvias_extremas = get_extreme_rain_data()

    if df_lluvias_extremas.empty:
        st.write("✅ No se detectaron lluvias extremas (>100 mm) desde 2018 hasta hoy.")
    else:
        st.write(f"🌧️ Total de días con lluvias extremas detectados: {len(df_lluvias_extremas)}")
        st.dataframe(df_lluvias_extremas[['time', 'ciudad', 'prcp', 'lat', 'lon']])

        # Mostrar el mapa con las lluvias extremas
        plot_extreme_rain_map(df_lluvias_extremas)

# ===================================
# FUNCIONES PARA SOLICITAR LA CLAVE DE AEMET
# ===================================
def get_weather_data(aemet_api_key):
    base_url = "https://opendata.aemet.es/opendata/api/observacion/convencional/todas/"
    headers = {"accept": "application/json"}

    try:
        response = requests.get(base_url, headers=headers, params={"api_key": aemet_api_key})
        response.raise_for_status()
        json_url = response.json()["datos"]

        data_response = requests.get(json_url)
        data_response.raise_for_status()
        weather_data = data_response.json()

        weather = []
        for obs in weather_data:
            weather.append({
                "fecha": obs["fint"],
                "latitud": obs["lat"],
                "longitud": obs["lon"],
                "temperatura": obs.get("ta", "N/A"),
                "humedad": obs.get("hr", "N/A"),
                "presion": obs.get("pres", "N/A"),
                "provincia": obs.get("ubi", "Desconocida")
            })

        return pd.DataFrame(weather)

    except Exception as e:
        st.error(f"Error al obtener datos meteorológicos de AEMET: {e}")
        return pd.DataFrame()

# Interfaz de entrada para la clave AEMET
aemet_api_key = st.text_input("Introduce la clave de API de AEMET para obtener datos meteorológicos (se pedirá solo al hacer clic)", type="password")

if aemet_api_key:
    weather_data = get_weather_data(aemet_api_key)

    if not weather_data.empty:
        st.write("Datos meteorológicos recientes:")
        st.dataframe(weather_data)
    else:
        st.warning("No se pudieron obtener datos meteorológicos.")


2025-03-04 18:38:31.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:38:31.332 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:38:31.333 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:38:31.334 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:38:31.337 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:38:31.338 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:38:31.339 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:38:31.340 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [9]:
import streamlit as st
import pandas as pd
import requests
import folium
from streamlit_folium import folium_static

# Funciones de obtención de datos reales

@st.cache_data(ttl=600)
def get_earthquake_data():
    url = "https://www.ign.es/web/resources/sismologia/sismos/provincia.json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        earthquakes = []
        for item in data:
            earthquakes.append({
                "fecha": item["fecha"],
                "hora": item["hora"],
                "magnitud": item["magnitud"],
                "profundidad": item["profundidad"],
                "latitud": item["latitud"],
                "longitud": item["longitud"],
                "localizacion": item["provincia"]
            })
        return pd.DataFrame(earthquakes)
    except Exception as e:
        st.error(f"Error al obtener datos de terremotos: {e}")
        return pd.DataFrame()


def get_weather_data(aemet_api_key):
    base_url = "https://opendata.aemet.es/opendata/api/observacion/convencional/todas/"
    headers = {"accept": "application/json"}

    try:
        response = requests.get(base_url, headers=headers, params={"api_key": aemet_api_key})
        response.raise_for_status()
        json_url = response.json()["datos"]

        data_response = requests.get(json_url)
        data_response.raise_for_status()
        weather_data = data_response.json()

        weather = []
        for obs in weather_data:
            weather.append({
                "fecha": obs["fint"],
                "latitud": obs["lat"],
                "longitud": obs["lon"],
                "temperatura": obs.get("ta", "N/A"),
                "humedad": obs.get("hr", "N/A"),
                "presion": obs.get("pres", "N/A"),
                "provincia": obs.get("ubi", "Desconocida")
            })

        return pd.DataFrame(weather)

    except Exception as e:
        st.error(f"Error al obtener datos meteorológicos de AEMET: {e}")
        return pd.DataFrame()


def plot_emergency_map():
    mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=6)

    terremotos = get_earthquake_data()
    for _, row in terremotos.iterrows():
        folium.CircleMarker(
            [row["latitud"], row["longitud"]],
            radius=5,
            color='red',
            fill=True,
            fill_opacity=0.7,
            popup=f"Terremoto {row['magnitud']}M en {row['localizacion']}"
        ).add_to(mapa)

    folium_static(mapa)


# Interfaz principal
st.title("Monitor de Emergencias en España")

# Mostrar mapa de terremotos al inicio, sin necesidad de claves
plot_emergency_map()

# Bloque opcional para obtener datos meteorológicos (solo cuando el usuario lo elige)
if st.button("Cargar datos meteorológicos de AEMET"):
    aemet_api_key = st.text_input("Introduce la clave de API de AEMET (se pedirá solo al hacer clic)", type="password")

    if aemet_api_key:
        weather_data = get_weather_data(aemet_api_key)

        if not weather_data.empty:
            st.write("Datos meteorológicos recientes:")
            st.dataframe(weather_data)
        else:
            st.warning("No se pudieron obtener datos meteorológicos.")
    else:
        st.warning("Introduce la clave para continuar.")


2025-03-04 18:35:29.745 No runtime found, using MemoryCacheStorageManager
2025-03-04 18:35:29.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:35:29.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:35:29.759 No runtime found, using MemoryCacheStorageManager
2025-03-04 18:35:29.760 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:35:29.761 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:35:29.762 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:35:30.268 Thread 'Thread-9': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-04 18:35:30.271 Thread 'Thread-9': missing ScriptRunContext! This warning can be ignored w

In [12]:
import folium
from folium import plugins

def create_base_map(center_coords, zoom_start=6):
    """Crea un mapa base de Folium con capas y controles."""
    m = folium.Map(location=center_coords, zoom_start=zoom_start, tiles="CartoDB positron")
    folium.LayerControl().add_to(m)
    return m


def add_earthquakes_to_map(m, earthquake_data):
    """Añade terremotos al mapa."""
    if earthquake_data.empty:
        return m

    group = folium.FeatureGroup(name="Terremotos 🌋")

    for _, quake in earthquake_data.iterrows():
        color = "red" if quake["magnitud"] >= 4 else "orange" if quake["magnitud"] >= 3 else "blue"
        popup_html = f"""
        <div>
            <strong>Terremoto M{quake['magnitud']}</strong><br>
            Fecha: {quake['fecha']}<br>
            Hora: {quake['hora']}<br>
            Magnitud: {quake['magnitud']}<br>
            Profundidad: {quake['profundidad']} km<br>
            Localización: {quake['localizacion']}
        </div>
        """
        folium.CircleMarker(
            [quake["latitud"], quake["longitud"]],
            radius=quake["magnitud"] * 2,
            color=color, fill=True, fill_opacity=0.7,
            popup=popup_html,
            tooltip=f"M{quake['magnitud']} - {quake['localizacion']}"
        ).add_to(group)

    group.add_to(m)
    return m


def add_fires_to_map(m, fire_data):
    """Añade incendios forestales al mapa."""
    if fire_data.empty:
        return m

    group = folium.FeatureGroup(name="Incendios 🔥")

    for _, fire in fire_data.iterrows():
        color = {"Activo": "red", "Controlado": "orange", "Extinguido": "green"}[fire["estado"]]
        popup_html = f"""
        <div>
            <strong>Incendio Forestal</strong><br>
            Fecha: {fire['fecha']}<br>
            Área afectada: {fire['area']} ha<br>
            Estado: {fire['estado']}<br>
            Provincia: {fire['provincia']}
        </div>
        """
        folium.CircleMarker(
            [fire["latitud"], fire["longitud"]],
            radius=min(15, fire["area"] / 2),
            color=color, fill=True, fill_opacity=0.7,
            popup=popup_html,
            tooltip=f"Incendio - {fire['provincia']} - {fire['estado']}"
        ).add_to(group)

    group.add_to(m)
    return m


def add_floods_to_map(m, flood_data):
    """Añade inundaciones al mapa."""
    if flood_data.empty:
        return m

    group = folium.FeatureGroup(name="Inundaciones 💧")

    for _, flood in flood_data.iterrows():
        color = {"Alto": "red", "Medio": "orange", "Bajo": "blue"}[flood["nivel"]]
        popup_html = f"""
        <div>
            <strong>Inundación</strong><br>
            Fecha: {flood['fecha']}<br>
            Nivel: {flood['nivel']}<br>
            Estado: {flood['estado']}<br>
            Provincia: {flood['provincia']}
        </div>
        """
        folium.CircleMarker(
            [flood["latitud"], flood["longitud"]],
            radius=10,
            color=color, fill=True, fill_opacity=0.7,
            popup=popup_html,
            tooltip=f"Inundación - {flood['provincia']} - {flood['nivel']}"
        ).add_to(group)

    group.add_to(m)
    return m


def add_storms_to_map(m, storm_data):
    """Añade temporales al mapa."""
    if storm_data.empty:
        return m

    group = folium.FeatureGroup(name="Temporales 🌪️")

    for _, storm in storm_data.iterrows():
        color = {"Muy fuerte": "purple", "Fuerte": "red", "Moderado": "orange"}[storm["intensidad"]]
        popup_html = f"""
        <div>
            <strong>Temporal</strong><br>
            Fecha: {storm['fecha']}<br>
            Tipo: {storm['tipo']}<br>
            Intensidad: {storm['intensidad']}<br>
            Provincia: {storm['provincia']}
        </div>
        """
        folium.CircleMarker(
            [storm["latitud"], storm["longitud"]],
            radius=12,
            color=color, fill=True, fill_opacity=0.7,
            popup=popup_html,
            tooltip=f"Temporal {storm['tipo']} - {storm['provincia']} - {storm['intensidad']}"
        ).add_to(group)

    group.add_to(m)
    return m


def add_flights_to_map(m, flight_data, show_commercial=True, show_emergency=True):
    """Añade vuelos comerciales y de emergencia al mapa."""
    if flight_data.empty:
        return m

    commercial_group = folium.FeatureGroup(name="Vuelos Comerciales ✈️")
    emergency_group = folium.FeatureGroup(name="Vuelos de Emergencia 🚁")

    for _, flight in flight_data.iterrows():
        icon = folium.Icon(icon="plane", prefix="fa", color="blue") if flight["type"] == "commercial" else folium.Icon(icon="helicopter", prefix="fa", color="red")
        group = commercial_group if flight["type"] == "commercial" else emergency_group
        popup_html = f"""
        <div>
            <strong>Vuelo {flight['callsign']}</strong><br>
            ICAO: {flight['icao24']}<br>
            Origen: {flight['origin']}<br>
            Destino: {flight.get('destination', 'N/A')}<br>
            Altitud: {flight['altitude']} ft<br>
            Velocidad: {flight['velocity']} km/h<br>
            Tipo: {"Comercial" if flight['type'] == 'commercial' else "Emergencia"}
        </div>
        """
        folium.Marker(
            [flight["latitude"], flight["longitude"]],
            icon=icon,
            popup=popup_html,
            tooltip=f"{flight['callsign']} - {flight['origin']}"
        ).add_to(group)

    if show_commercial:
        commercial_group.add_to(m)
    if show_emergency:
        emergency_group.add_to(m)

    return m


def add_weather_to_map(m, weather_data, show_weather=True):
    """Añade condiciones meteorológicas al mapa."""
    if weather_data.empty or not show_weather:
        return m

    group = folium.FeatureGroup(name="Condiciones Meteorológicas 🌤️")

    icon_map = {
        "Soleado": "sun", "Parcialmente nublado": "cloud-sun",
        "Nublado": "cloud", "Lluvia ligera": "cloud-rain",
        "Lluvia fuerte": "cloud-showers-heavy", "Tormenta": "bolt",
        "Nieve": "snowflake", "Niebla": "smog"
    }

    for _, weather in weather_data.iterrows():
        icon = folium.Icon(icon=icon_map.get(weather["condicion"], "cloud"), prefix="fa", color="green")
        popup_html = f"""
        <div>
            <strong>{weather['ciudad']}</strong><br>
            Temperatura: {weather['temperatura']}°C<br>
            Condición: {weather['condicion']}<br>
            Humedad: {weather['humedad']}%<br>
            Viento: {weather['velocidad_viento']} km/h {weather['direccion_viento']}
        </div>
        """
        folium.Marker([weather["latitud"], weather["longitud"]], icon=icon, popup=popup_html).add_to(group)

    group.add_to(m)
    return m


In [13]:
import folium
from folium import plugins

def create_base_map(center_coords, zoom_start=6):
    """Crea un mapa base de Folium centrado en las coordenadas proporcionadas"""
    m = folium.Map(location=center_coords, zoom_start=zoom_start, tiles="CartoDB positron")
    folium.LayerControl().add_to(m)
    return m


def add_earthquakes_to_map(m, earthquake_data, show=True):
    if not show or earthquake_data.empty:
        return m

    group = folium.FeatureGroup(name="Terremotos 🌋")

    for _, quake in earthquake_data.iterrows():
        color = "red" if quake["magnitud"] >= 4.0 else "orange" if quake["magnitud"] >= 3.0 else "blue"

        popup_html = f"""
        <div style="width:200px">
            <h4>Terremoto M{quake['magnitud']}</h4>
            <b>Fecha:</b> {quake['fecha']}<br>
            <b>Hora:</b> {quake['hora']}<br>
            <b>Profundidad:</b> {quake['profundidad']} km<br>
            <b>Intensidad:</b> {quake['intensidad']}<br>
            <b>Localización:</b> {quake['localizacion']}
        </div>
        """

        folium.CircleMarker(
            location=[quake["latitud"], quake["longitud"]],
            radius=quake["magnitud"] * 2,
            color=color,
            fill=True, fill_opacity=0.7,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"M{quake['magnitud']} - {quake['localizacion']}"
        ).add_to(group)

    group.add_to(m)
    return m


def add_fires_to_map(m, fire_data, show=True):
    if not show or fire_data.empty:
        return m

    group = folium.FeatureGroup(name="Incendios 🔥")

    for _, fire in fire_data.iterrows():
        color = "red" if fire["estado"] == "Activo" else "orange" if fire["estado"] == "Controlado" else "green"

        popup_html = f"""
        <div style="width:200px">
            <h4>Incendio Forestal</h4>
            <b>Fecha:</b> {fire['fecha']}<br>
            <b>Área:</b> {fire['area']} ha<br>
            <b>Estado:</b> {fire['estado']}<br>
            <b>Provincia:</b> {fire['provincia']}
        </div>
        """

        folium.CircleMarker(
            location=[fire["latitud"], fire["longitud"]],
            radius=min(15, fire["area"] / 2),
            color=color,
            fill=True, fill_opacity=0.7,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"Incendio - {fire['provincia']} - {fire['estado']}"
        ).add_to(group)

    group.add_to(m)
    return m


def add_floods_to_map(m, flood_data, show=True):
    if not show or flood_data.empty:
        return m

    group = folium.FeatureGroup(name="Inundaciones 💧")

    for _, flood in flood_data.iterrows():
        color = "red" if flood["nivel"] == "Alto" else "orange" if flood["nivel"] == "Medio" else "blue"

        popup_html = f"""
        <div style="width:200px">
            <h4>Inundación</h4>
            <b>Fecha:</b> {flood['fecha']}<br>
            <b>Nivel:</b> {flood['nivel']}<br>
            <b>Estado:</b> {flood['estado']}<br>
            <b>Provincia:</b> {flood['provincia']}
        </div>
        """

        folium.CircleMarker(
            location=[flood["latitud"], flood["longitud"]],
            radius=10,
            color=color,
            fill=True, fill_opacity=0.7,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"Inundación - {flood['provincia']} - {flood['nivel']}"
        ).add_to(group)

    group.add_to(m)
    return m


def add_storms_to_map(m, storm_data, show=True):
    if not show or storm_data.empty:
        return m

    group = folium.FeatureGroup(name="Temporales 🌪️")

    for _, storm in storm_data.iterrows():
        color = "purple" if storm["intensidad"] == "Muy fuerte" else "red" if storm["intensidad"] == "Fuerte" else "orange"

        popup_html = f"""
        <div style="width:200px">
            <h4>Temporal</h4>
            <b>Fecha:</b> {storm['fecha']}<br>
            <b>Tipo:</b> {storm['tipo']}<br>
            <b>Intensidad:</b> {storm['intensidad']}<br>
            <b>Provincia:</b> {storm['provincia']}
        </div>
        """

        folium.CircleMarker(
            location=[storm["latitud"], storm["longitud"]],
            radius=12,
            color=color,
            fill=True, fill_opacity=0.7,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"Temporal {storm['tipo']} - {storm['provincia']} - {storm['intensidad']}"
        ).add_to(group)

    group.add_to(m)
    return m


def add_flights_to_map(m, flight_data, show_commercial=True, show_emergency=True):
    if flight_data.empty:
        return m

    commercial_group = folium.FeatureGroup(name="Vuelos Comerciales ✈️")
    emergency_group = folium.FeatureGroup(name="Vuelos de Emergencia 🚁")

    for _, flight in flight_data.iterrows():
        icon = folium.Icon(icon="plane", prefix="fa", color="blue") if flight["type"] == "commercial" else folium.Icon(icon="helicopter", prefix="fa", color="red")
        group = commercial_group if flight["type"] == "commercial" else emergency_group

        popup_html = f"""
        <div style="width:200px">
            <h4>Vuelo {flight['callsign']}</h4>
            <b>Altitud:</b> {flight['altitude']} ft<br>
            <b>Velocidad:</b> {flight['velocity']} km/h<br>
            <b>Tipo:</b> {'Comercial' if flight['type'] == 'commercial' else 'Emergencia'}
        </div>
        """

        folium.Marker(
            location=[flight["latitude"], flight["longitude"]],
            icon=icon,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=flight['callsign']
        ).add_to(group)

    if show_commercial:
        commercial_group.add_to(m)
    if show_emergency:
        emergency_group.add_to(m)

    return m


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.231.218:8501

⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://angry-crabs-read.loca.lt
y
